In [1]:
%run read_data.py
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import time
import scipy.optimize as op
import george as gg
import george.kernels as kr

In [56]:
def gen_images_with_GP(file_name, path_to_read, path_to_export,error_margin, points, show_fig):
    #file_name must be like: DES_SN123456.DAT
    file_id = file_name[4:12]
    file_read = read_sn(path_to_read + file_name)
    sn_type = file_read['SIM_COMMENT'][3]
    #TODO, add verification here, add type verification and also if it has 4 bands, 
#if it does not, return 0
    GP_Function(file_read['df'], file_id, sn_type, path_to_export,error_margin, points, show_fig)

def GP_Function(df_all, file_id, sn_type, path_to_export,error_margin, points, show_fig):
    
    x = np.linspace(0, df_all.MJD.max(), 200)
    
    data = {band: df[['MJD', 'FLUXCAL', 'FLUXCALERR']].values for band, df in df_all.groupby('FLT')}
    colors = {'g': 'g', 'r': 'r', 'i': 'y', 'z': 'b'}

    for band, dat in data.items():
        gp = gg.GP((500**2)*kr.ExpSquaredKernel(metric=20**2), fit_mean=True)
        gp.compute(dat[:,0], dat[:,2])

        def nll(p):
            gp.set_parameter_vector(p)
            ll = gp.log_likelihood(dat[:,1], quiet=True)
            return -ll if np.isfinite(ll) else 1e25
        
        def grad_nll(p):
            gp.set_parameter_vector(p)
            return -gp.grad_log_likelihood(dat[:,1], quiet=True)
            
        p0 = gp.get_parameter_vector()
        results = op.minimize(nll, p0, jac=grad_nll, method="L-BFGS-B")
          
        mu, var = gp.predict(dat[:,1], x, return_var=True)
        std = np.sqrt(var)
        
        fig = plt.figure(num=None, figsize=(8, 5), dpi=8, facecolor='w', edgecolor='w')
        #plt.ylim(bottom = 1)
        #plt.xlim(-0, 250)
        if error_margin == True:
            plt.fill_between(x, mu-std, mu+std, alpha=.1, color=colors[band[-1]])
        #plt.tight_layout()
        plt.plot(x, mu, color= 'k', linewidth=5)
        if points == True:
            plt.errorbar(dat[:,0], dat[:,1], yerr=dat[:,2], fmt='o', color= 'k')
        plt.axis('off')
        plt.savefig(path_to_export + file_id + '_'+ sn_type +'_'+ band + '.png', bbox_inches='tight')
        if show_fig == False:
            plt.close(fig)

In [58]:
#single test
file_name = 'DES_SN067538.DAT'
path_to_read = 'data/'
path_to_export = 'teste/'
read = read_sn(path_to_read + file_name)

#file_id = file_name[4:12]
#sn_type = read['SIM_COMMENT'][3]

gen_images_with_GP(file_name, path_to_read, path_to_export, 
                   error_margin = False, points = False, show_fig = False)